In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 0.0, 1.0
p_train = np.random.uniform(p_low, p_high, (train_size, 2)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 2)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 2)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train}, name="train")
data_test = DictDataset({"p":p_test}, name="test")
data_dev = DictDataset({"p":p_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic()

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    model.set_param_val({"p":p})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/qp_exact.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:12<00:00, 13.71it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.301369          0.287015      0.072262
std       0.015466          0.665298      0.031805
min      -0.310000          0.000000      0.056964
25%      -0.310000          0.000000      0.060942
50%      -0.310000          0.000000      0.061737
75%      -0.300000          0.000000      0.062751
max      -0.212920          3.460264      0.184068
Number of infeasible solution: 212


## Heuristic

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    model.set_param_val({"p":p})
    model_rel = model.relax()
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/qp_heur.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:15<00:00, 13.27it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.302556          0.338047      0.070023
std       0.015731          0.694741      0.031190
min      -0.310000          0.000000      0.053624
25%      -0.310000          0.000000      0.057207
50%      -0.310000          0.000000      0.058233
75%      -0.300000          0.231860      0.068860
max      -0.212920          3.460264      0.182869
Number of infeasible solution: 335


## Learnable Rounding

In [10]:
# hyperparameters
penalty_weight = 10   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 3       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-2             # learning rate
batch_size = 64       # batch size

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# define quadratic objective functions and constraints for both problem types
obj_rel, constrs_rel = nmQuadratic(["x"], ["p"], penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmQuadratic(["x_rnd"], ["p"], penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=2, outsize=4, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=6, hidden_dims=[hsize]*hlayers_rnd, output_dim=4)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p"], var_keys=["x"], output_keys=["x_rnd"],
                       int_ind={"x":[2,3]}, continuous_update=False, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [12]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 4.2779388427734375
epoch: 1  train_loss: 3.1776251792907715
epoch: 2  train_loss: 3.3425543308258057
epoch: 3  train_loss: 2.1936593055725098
epoch: 4  train_loss: 1.9622385501861572
epoch: 5  train_loss: 1.9493381977081299
epoch: 6  train_loss: 2.089452028274536
epoch: 7  train_loss: 2.0913796424865723
epoch: 8  train_loss: 2.0661263465881348
epoch: 9  train_loss: 2.1217825412750244
epoch: 10  train_loss: 2.098419189453125
epoch: 11  train_loss: 2.0379459857940674
epoch: 12  train_loss: 2.140923500061035
epoch: 13  train_loss: 1.977126121520996
epoch: 14  train_loss: 2.1311705112457275
epoch: 15  train_loss: 2.0974972248077393
epoch: 16  train_loss: 2.157132387161255
epoch: 17  train_loss: 2.0126335620880127
epoch: 18  train_loss: 2.07010817527771
epoch: 19  train_loss: 2.150472640991211
epoch: 20  train_loss: 2.078364849090576
epoch: 21  train_loss: 2.1132590770721436
epoch: 22  train_loss: 2.147148847579956
epoch: 23  train_loss: 2.1320548057556152
epoch: 24  t

<All keys matched successfully>

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(4):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/qp_nm.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 132.24it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.249774          0.145520      0.006935
std       0.069338          0.359052      0.001169
min      -0.388125          0.000000      0.005000
25%      -0.281723          0.000000      0.006084
50%      -0.277095          0.000000      0.006600
75%      -0.244082          0.000000      0.007408
max       0.166659          1.572065      0.013440
Number of infeasible solution: 212
